# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 6 2022 2:00PM,248077,10,0086057523,ISDIN Corporation,Released
1,Oct 6 2022 2:00PM,248077,10,0086057535,ISDIN Corporation,Released
2,Oct 6 2022 2:00PM,248077,10,0086057541,ISDIN Corporation,Released
3,Oct 6 2022 2:00PM,248077,10,0086057548,ISDIN Corporation,Released
4,Oct 6 2022 2:00PM,248077,10,0086057559,ISDIN Corporation,Released
5,Oct 6 2022 2:00PM,248077,10,0086058764,ISDIN Corporation,Released
6,Oct 6 2022 2:00PM,248077,10,0086059760,ISDIN Corporation,Released
7,Oct 6 2022 2:00PM,248077,10,0086059761,ISDIN Corporation,Released
8,Oct 6 2022 2:00PM,248077,10,0086060084,ISDIN Corporation,Released
9,Oct 6 2022 2:00PM,248077,10,0086060085,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,248070,Executing,19
23,248071,Released,1
24,248072,Released,1
25,248076,Released,33
26,248077,Released,20


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248070,NaN,19.0,NaN
248071,NaN,NaN,1.0
248072,NaN,NaN,1.0
248076,NaN,NaN,33.0
248077,NaN,NaN,20.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247985,0.0,0.0,3.0
247991,0.0,0.0,1.0
248000,6.0,4.0,10.0
248031,0.0,0.0,12.0
248037,3.0,4.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247985,0,0,3
247991,0,0,1
248000,6,4,10
248031,0,0,12
248037,3,4,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247985,0,0,3
1,247991,0,0,1
2,248000,6,4,10
3,248031,0,0,12
4,248037,3,4,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247985,,,3
1,247991,,,1
2,248000,6,4,10
3,248031,,,12
4,248037,3,4,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 6 2022 2:00PM,248077,10,ISDIN Corporation
20,Oct 6 2022 2:00PM,248076,10,ISDIN Corporation
53,Oct 6 2022 1:39PM,248072,19,"AdvaGen Pharma, Ltd"
54,Oct 6 2022 1:37PM,248071,20,"ACI Healthcare USA, Inc."
55,Oct 6 2022 1:22PM,248070,19,"AdvaGen Pharma, Ltd"
74,Oct 6 2022 12:50PM,248067,19,"GUSA Granules USA, Inc."
75,Oct 6 2022 12:35PM,248064,19,"GUSA Granules USA, Inc."
76,Oct 6 2022 12:23PM,248061,16,Sartorius Bioprocess Solutions
77,Oct 6 2022 12:11PM,248059,10,"Methapharm, Inc."
87,Oct 6 2022 11:36AM,248052,16,"SHL Pharma, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 6 2022 2:00PM,248077,10,ISDIN Corporation,,,20
1,Oct 6 2022 2:00PM,248076,10,ISDIN Corporation,,,33
2,Oct 6 2022 1:39PM,248072,19,"AdvaGen Pharma, Ltd",,,1
3,Oct 6 2022 1:37PM,248071,20,"ACI Healthcare USA, Inc.",,,1
4,Oct 6 2022 1:22PM,248070,19,"AdvaGen Pharma, Ltd",,19,
5,Oct 6 2022 12:50PM,248067,19,"GUSA Granules USA, Inc.",,,1
6,Oct 6 2022 12:35PM,248064,19,"GUSA Granules USA, Inc.",,,1
7,Oct 6 2022 12:23PM,248061,16,Sartorius Bioprocess Solutions,,,1
8,Oct 6 2022 12:11PM,248059,10,"Methapharm, Inc.",,,10
9,Oct 6 2022 11:36AM,248052,16,"SHL Pharma, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 2:00PM,248077,10,ISDIN Corporation,20,,
1,Oct 6 2022 2:00PM,248076,10,ISDIN Corporation,33,,
2,Oct 6 2022 1:39PM,248072,19,"AdvaGen Pharma, Ltd",1,,
3,Oct 6 2022 1:37PM,248071,20,"ACI Healthcare USA, Inc.",1,,
4,Oct 6 2022 1:22PM,248070,19,"AdvaGen Pharma, Ltd",,19,
5,Oct 6 2022 12:50PM,248067,19,"GUSA Granules USA, Inc.",1,,
6,Oct 6 2022 12:35PM,248064,19,"GUSA Granules USA, Inc.",1,,
7,Oct 6 2022 12:23PM,248061,16,Sartorius Bioprocess Solutions,1,,
8,Oct 6 2022 12:11PM,248059,10,"Methapharm, Inc.",10,,
9,Oct 6 2022 11:36AM,248052,16,"SHL Pharma, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 2:00PM,248077,10,ISDIN Corporation,20,,
1,Oct 6 2022 2:00PM,248076,10,ISDIN Corporation,33,,
2,Oct 6 2022 1:39PM,248072,19,"AdvaGen Pharma, Ltd",1,,
3,Oct 6 2022 1:37PM,248071,20,"ACI Healthcare USA, Inc.",1,,
4,Oct 6 2022 1:22PM,248070,19,"AdvaGen Pharma, Ltd",,19,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 2:00PM,248077,10,ISDIN Corporation,20.0,NaN,NaN
1,Oct 6 2022 2:00PM,248076,10,ISDIN Corporation,33.0,NaN,NaN
2,Oct 6 2022 1:39PM,248072,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
3,Oct 6 2022 1:37PM,248071,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
4,Oct 6 2022 1:22PM,248070,19,"AdvaGen Pharma, Ltd",NaN,19.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 2:00PM,248077,10,ISDIN Corporation,20.0,0.0,0.0
1,Oct 6 2022 2:00PM,248076,10,ISDIN Corporation,33.0,0.0,0.0
2,Oct 6 2022 1:39PM,248072,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
3,Oct 6 2022 1:37PM,248071,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
4,Oct 6 2022 1:22PM,248070,19,"AdvaGen Pharma, Ltd",0.0,19.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2232362,90.0,17.0,0.0
15,496037,11.0,8.0,9.0
16,496113,2.0,0.0,0.0
19,1736412,6.0,19.0,0.0
20,248071,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2232362,90.0,17.0,0.0
1,15,496037,11.0,8.0,9.0
2,16,496113,2.0,0.0,0.0
3,19,1736412,6.0,19.0,0.0
4,20,248071,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,90.0,17.0,0.0
1,15,11.0,8.0,9.0
2,16,2.0,0.0,0.0
3,19,6.0,19.0,0.0
4,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,90.0
1,15,Released,11.0
2,16,Released,2.0
3,19,Released,6.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,0.0,9.0,0.0,0.0,0.0
Executing,17.0,8.0,0.0,19.0,0.0
Released,90.0,11.0,2.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,0.0,9.0,0.0,0.0,0.0
1,Executing,17.0,8.0,0.0,19.0,0.0
2,Released,90.0,11.0,2.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,0.0,9.0,0.0,0.0,0.0
1,Executing,17.0,8.0,0.0,19.0,0.0
2,Released,90.0,11.0,2.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()